Dan Shea  
2021-06-20  

##### Problem
For a collection of strings, a larger string containing every one of the smaller strings as a substring is called a superstring.

By the assumption of parsimony, a shortest possible superstring over a collection of reads serves as a candidate chromosome.

__Given:__ At most 50 DNA strings of approximately equal length, not exceeding 1 kbp, in FASTA format (which represent reads deriving from the same strand of a single linear chromosome).

The dataset is guaranteed to satisfy the following condition: there exists a unique way to reconstruct the entire chromosome from these reads by gluing together pairs of reads that overlap by more than half their length.

__Return:__ A shortest superstring containing all the given strings (thus corresponding to a reconstructed chromosome).

##### Sample Dataset
```
>Rosalind_56
ATTAGACCTG
>Rosalind_57
CCTGCCGGAA
>Rosalind_58
AGACCTGCCG
>Rosalind_59
GCCGGAATAC
```
##### Sample Output
```
ATTAGACCTGCCGGAATAC
```

In [1]:
from Bio import SeqIO
from functools import reduce
import networkx as nx
from collections import deque
import re

In [2]:
def parse_fasta(filename):
    with open(filename, 'r') as fh:
        seqio = SeqIO.parse(fh, 'fasta')
        return list(seqio)

def create_lookup(seqs):
    lookup = {}
    for seq in seqs:
        lookup.update({seq.name: seq.seq})
    return lookup

def kmer_analysis(seqs):
    # generate all kmers 50% or greater than read length for all reads
    left_read_kmers = {seq.name: set() for seq in seqs}
    right_read_kmers = {seq.name: set() for seq in seqs}
    all_kmers = set()
    for seq in seqs:
        N = len(seq.seq) // 2
        for i,j in zip(range(N, len(seq.seq)-1),range(N, 0, -1)):
            left_read_kmers[seq.name].add(seq.seq[0:i+1])
            right_read_kmers[seq.name].add(seq.seq[j:])
        all_kmers.update(left_read_kmers[seq.name])
        all_kmers.update(right_read_kmers[seq.name])
    return left_read_kmers, right_read_kmers, all_kmers

def find_longest_overlaps(seqs, left_read_kmers, right_read_kmers):
    overlaps = {seq.name: dict() for seq in seqs}
    for i in range(len(seqs)-1):
        for j in range(i+1, len(seqs)):
            A = seqs[i].name
            B = seqs[j].name
            # Left of A overlap Right of B
            overlap = left_read_kmers[A].intersection(right_read_kmers[B])
            if overlap:
                overlap = max(overlap, key=lambda x: len(x))
            else:
                overlap = None
            overlaps[B][A] = overlap
            
            # Right of A overlap Left of A
            overlap = left_read_kmers[B].intersection(right_read_kmers[A])
            if overlap:
                overlap = max(overlap, key=lambda x: len(x))
            else:
                overlap = None
            overlaps[A][B] = overlap
            
    return overlaps

def prune_orphans(overlaps):
    # We must iterate over a copy in order to modify the actual dict
    for A in overlaps.copy():
        for B in overlaps[A].copy():
            if overlaps[A][B] is None:
                overlaps[A].pop(B, None)
        if overlaps[A] == {}:
            overlaps.pop(A, None)

def construct_sequence(overlaps, lookup):
    # Gather all of the directed edges that will construct the De Bruijn Graph    
    edges = [(a, b) for a in overlaps for b in overlaps[a]]
    
    # Create the De Bruijn Graph
    DeBruijnGraph = nx.DiGraph()
    
    # Add all of the nodes to the Graph
    DeBruijnGraph.add_edges_from(edges)
    
    # If there is an eulerian path, get that path
    if nx.algorithms.has_eulerian_path(DeBruijnGraph):
        walk = list(nx.algorithms.eulerian_path(DeBruijnGraph))
    else:
        # Not going to throw an exception, just return None for now
        return None
    
    sequence = str(lookup[walk[0][0]])
    for e in walk:
        skip = len(overlaps[e[0]][e[1]])
        sequence += str(lookup[e[1]])[skip:]
    
    return sequence

def parse_file_print_ans(filename):
    reads = parse_fasta(filename)
    lookup = create_lookup(reads)
    left_read_kmers, right_read_kmers, all_kmers = kmer_analysis(reads)
    overlaps = find_longest_overlaps(reads, left_read_kmers, right_read_kmers)
    prune_orphans(overlaps)
    print(construct_sequence(overlaps, lookup))

In [3]:
parse_file_print_ans('sample.txt')

ATTAGACCTGCCGGAATAC


In [4]:
parse_file_print_ans('rosalind_long.txt')

AAAGCGCATCTGTCTTGTGTTCGTACCTTTTCTATGCTCACACTTGTGGTATCTAAAATTGGTCCTATGGTAATGCGATCCATATGTGGTGTACACTGATCCGGACCTGTTCATCCTGCGTGCTTCCCTGCTGCTCACAGGCGGATACCGTGTTTCTCATCCGGCATTTGACTCACATCACAGGGGGCCGACACATCAATCAGGCGCGCTATACTATTGTTGGCGCCAGAGGGCTCAAAGCACGGCGTTGATGATGAGTACATGATTACTGCAAGTCGGTAGTTACAAAACTTTTACGGATTCTCCCAATGGCCAGACAAGAGCGCCGTCTTAAGGTAGGGAAATATCGTATTCAGAATCGATATCTGGCAACAGCATCCTCTACATCTGAATACGATTGATGGCCGAGCCTCGTCAACCGAGACTATCACGTAGTTGTCATTCATGCGCCGATAGCGGGCTTAACCCTAATGATAGAGTGACTCCCGTTGAAATAAGCGTTCGTCTGTGATGCTCCCGGTGGCTGGCCTGTATTTCCAGTGTCGCGATTAAGTGGGTGACTAAGGCACACACGGGACAGACTACGATCGACTTGCATGTTTCCAGGGCATGTATCCGCCGCGACGCTTTGCCTTAAGTACCGCTAAGTTTAGTGTAGCCTAGTAACTCACACAGGATGCCACGGAACTTCCTGGTCCTTCTTGAAAATCTAAACCCACAAGTACTGGTTCCCGCGTCATTCTATTTAAAAATAGACGACCGAAAATTTAGCCCATAGTTAGTCTTAGCTAGTTAAGATTACCTAACTAATCGATCAGTGTTTAGCAAATCAACGCTCATTATCTTTTGATCGTAGATATTCCAGGGGGCAGCAGTACCTGCAATAGCAGGGCGTATACCGGCGTGGCGCCATACTCGAATAAATGCGTCGCACCTACGGGTTAGTCCTGGTGGGGTTATAAGCTTACCTTCGTGCACTTTAACGTGAGTGACACCTGGG

#### Discussion
We can assemble the reads by finding the overlaps and constructing a De Bruijn graph. (See: https://en.wikipedia.org/wiki/De_Bruijn_graph)  
We then perform a walk of the Eulerian path of the graph re-constructs the sequence. (See: https://en.wikipedia.org/wiki/Eulerian_path)  